# Music Genre Classification using Neural Network
This notebook builds a neural network to classify music genres based on MFCC data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import platform

import utils

In [ ]:
# Enable GPU acceleration (Metal for macOS, CUDA for PC, fallback to CPU)
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        if tf.test.is_built_with_cuda():
            print(f"CUDA is enabled. Using GPU(s) with CUDA: {physical_devices}")
        elif platform.system() == 'Darwin':
            print(f"Metal is enabled. Using Metal-enabled GPU on macOS: {physical_devices}")
        else:
            print(f"GPU detected but neither CUDA nor Metal is enabled: {physical_devices}")
    except RuntimeError as e:
        print(f"Error enabling GPU acceleration: {e}")
else:
    print("No GPU found. Running on CPU.")

In [ ]:
# Load MFCC data
mfcc_df = utils.load_mfcc_data()
labels = pd.read_csv('project_data/tracks.csv')['genre']

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# Drop non-numeric columns
mfcc_df = mfcc_df.drop(columns=['file_name'])

# Flatten multi-index columns and scale features
mfcc_df.columns = ['_'.join(map(str, col)) for col in mfcc_df.columns]
scaler = StandardScaler()
X = scaler.fit_transform(mfcc_df)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Add batch normalization and modify the neural network
model = Sequential([
    Dense(256, input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation='softmax')
])

# Compile the model with a learning rate scheduler
initial_lr = 0.001
lr_schedule = LearningRateScheduler(lambda epoch: initial_lr * 0.95 ** epoch)

model.compile(optimizer=Adam(learning_rate=initial_lr),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=100,
                    validation_split=0.2,
                    callbacks=[lr_schedule, early_stopping])

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Visualize training results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Display test data and predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

for i in range(10):  # Display the first 10 test samples
    print(f"Test Sample {i}: True Label = {label_encoder.inverse_transform([y_test[i]])[0]}, Predicted Label = {label_encoder.inverse_transform([predicted_classes[i]])[0]}")